### 基礎設定

In [2]:
import os
from dotenv import load_dotenv 
from langchain_openai import ChatOpenAI

load_dotenv(override=True)

from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o")

#### 1. Documents 與 Document Loaders

在 LangChain 裡面，有個很核心的概念叫做 `Document`（文件）。

簡單來說，你就把它想像成是一個**容器**或**物件**，專門用來裝「一段文字」以及「關於這段文字的補充資訊」。

它主要有三個屬性（欄位）：

* **`page_content`**：
    這就是**內容本身**啦！格式是字串（string），也就是你真正要處理的那段文字。
* **`metadata`**：
    這是一個字典（dict），讓你放**補充資訊**用的。你想塞什麼都可以，比如這段文字是從哪個網址抓下來的？作者是誰？或是它跟其他文件有什麼關聯？通通都可以記在這裡。
* **`id`**：（選填）
    就是這個文件的 ID，有點像身分證字號，用來辨識它用的（也是字串）。

> **💡 重點筆記：**
> 雖然這個物件叫做 `Document`，但千萬不要以為它一定是一整份 PDF 或一整本書喔！
>
> 在實務上，一個 `Document` 物件通常只代表一大份文件裡切分出來的**「一小塊片段」**（chunk）。

In [3]:
from langchain_core.documents import Document

documents = [
    Document(
        page_content="Dogs are great companions, known for their loyalty and friendliness.",
        metadata={"source": "mammal-pets-doc"},
    ),
    Document(
        page_content="Cats are independent pets that often enjoy their own space.",
        metadata={"source": "mammal-pets-doc"},
    ),
]

接著我們來講講怎麼把檔案讀進來（Loading documents）。

我們要試著把一個 PDF 檔讀進去，轉成一連串的 Document 物件。這裡用 飲食聖經/你也可以用自己的啦 PDF。如果你想知道還有哪些工具可以用，隨時都可以去查 LangChain 的官方文件，裡面有列出各種 PDF 載入器（Loaders）。

以 PyPDFLoader 這個工具來說，它的邏輯是 「PDF 的每一頁，都會轉成一個獨立的 Document 物件」。

對於轉換出來的每一個 Document，我們都可以很輕鬆地拿到以下資訊：

頁面內容：就是那一頁上面的文字（字串格式）。

Metadata：裡面會紀錄檔名以及它是第幾頁。

In [4]:
from langchain_community.document_loaders import PyPDFLoader

file_path = "example_data/diet.pdf"
loader = PyPDFLoader(file_path)

docs = loader.load()

print(len(docs))

Ignoring wrong pointing object 921 0 (offset 0)
Ignoring wrong pointing object 1022 0 (offset 0)
Ignoring wrong pointing object 1033 0 (offset 0)
Ignoring wrong pointing object 1040 0 (offset 0)


272


In [5]:
print(f"{docs[1].page_content[:200]}\n")
print(docs[1].metadata)

About the Authors 
Dr. Mike Israetel​​ holds a PhD in Sport Physiology, and is currently the head science consultant for Renaissance 
Periodization. Mike was formerly a professor of Exercise and Sport

{'producer': 'iOS Version 16.3 (Build 20D47) Quartz PDFContext', 'creator': 'PyPDF', 'creationdate': "D:20230222113409Z00'00'", 'moddate': "D:20230222113409Z00'00'", 'source': 'example_data/diet.pdf', 'total_pages': 272, 'page': 1, 'page_label': '2'}


#### 2. **Splitting (切分)**

對於「資料檢索」或「回答問題」這類應用來說，如果直接用「一整頁 PDF」當作單位，顆粒度通常**太粗了**（太大了）。

我們的最終目標是：使用者問什麼，我們就抓出精準能回答該問題的 `Document` 物件。如果把 PDF 切得更細一點，就能確保真正相關的關鍵內容，意思不會被旁邊一大堆不相關的雜訊文字給淹沒（Washed out）。

這時候就要用 **Text Splitters**（文字切割器）來處理：

* **怎麼切？**
    這裡示範一個簡單的切法，依照「字元數」來切。
    我們設定每塊 (Chunk) 是 **1000 個字元**，而且每塊之間要有 **200 個字元的 Overlap (重疊)**。
    > 
* **為什麼要重疊 (Overlap)？**
    這是為了避免切的時候剛好「手起刀落」把一句完整的話切斷，或者是把關鍵句跟它的上下文硬生生分家，導致語意不連貫。

* **用什麼工具？**
    我們使用 `RecursiveCharacterTextSplitter`。
    這支工具很聰明，它會遞迴地嘗試用常見的分隔符號（像是換行符號 `\n`）來切，直到切出來的每一塊大小都符合規定。這通常也是處理一般文字時，**官方最推薦**的做法。

* **額外設定：**
    我們把 `add_start_index` 設為 `True`。
    這樣做的好處是，切分出來的每一個小 `Document`，它的 metadata 裡都會多一個 `"start_index"` 欄位，用來紀錄這段文字原本是在「原始文件」的第幾個字元開始的（保留它的原始位置資訊）。

In [6]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000, chunk_overlap=200, add_start_index=True
)
all_splits = text_splitter.split_documents(docs)

print(len(all_splits))

1022


In [7]:
all_splits[0]

Document(metadata={'producer': 'iOS Version 16.3 (Build 20D47) Quartz PDFContext', 'creator': 'PyPDF', 'creationdate': "D:20230222113409Z00'00'", 'moddate': "D:20230222113409Z00'00'", 'source': 'example_data/diet.pdf', 'total_pages': 272, 'page': 1, 'page_label': '2', 'start_index': 0}, page_content='About the Authors \nDr. Mike Israetel\u200b\u200b holds a PhD in Sport Physiology, and is currently the head science consultant for Renaissance \nPeriodization. Mike was formerly a professor of Exercise and Sport Science in the School of Public Health at Temple \nUniversity in Philadelphia, where he taught several courses, including Nutrition for Public Health, Advanced Sports \nNutrition and Exercise, and Nutrition and Behavior. He has worked as a consultant on sports nutrition to the U.S. \nOlympic Training Site in Johnson City, TN, and has been an invited speaker at numerous scientific and \nperformance/health conferences, including nutritional seminars at the U.S. Olympic Training Cent

#### 3. Vector stores (向量資料庫)

接下來講 **VectorStore** 物件。

在 LangChain 裡，這個物件主要負責兩件事：把文字或 `Document` 物件**存進去**，以及用各種「相似度演算法」把相關的內容**查出來**。

在初始化 VectorStore 的時候，通常都需要搭配一個 **Embedding Model (嵌入模型)**。這個模型的功能，就是負責把人類看的文字，轉換成電腦懂的「數字向量」。

LangChain 已經幫我們整合（Integrate）了一大堆市面上常見的向量資料庫技術。大致可以分成這幾類：

* **雲端託管型 (Hosted by provider)**：
    像是一些雲端大廠提供的服務，這種通常需要設定 API Key 或帳號密碼才能用。
* **獨立架構型 (Separate infrastructure)**：
    像是 **Postgres** 這種，需要自己架設伺服器或是用第三方服務，可以跑在本機也可以跑在雲端。
* **記憶體型 (In-memory)**：
    資料直接存在記憶體裡面，這種最適合拿來跑輕量級的任務或是做測試。

既然有這麼多選擇，我們接下來就挑一個來試試看吧！

In [8]:
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

In [9]:
vector_1 = embeddings.embed_query(all_splits[0].page_content)
vector_2 = embeddings.embed_query(all_splits[1].page_content)

assert len(vector_1) == len(vector_2)
print(f"Generated vectors of length {len(vector_1)}\n")
print(vector_1[:10])

Generated vectors of length 1536

[0.007661295123398304, -0.005888405721634626, -0.0047992016188800335, 0.006364304106682539, -0.00877395924180746, 0.011052909307181835, -0.0030045281164348125, 0.014746150001883507, 0.012829150073230267, -0.058930978178977966]


當我們的 Vector Store 實體化（建立）出來之後，接下來就可以把文件拿來「建索引」（Index），也就是把資料塞進去啦。

要注意喔，大多數 Vector Store 的實作都支援讓你直接連線到一個「現有」的資料庫——通常你只要提供 Client、Index 名稱或其他相關資訊就能連上了。

一旦 Vector Store 準備好，而且文件也都存進去了，我們就可以開始對它進行「查詢」（Query）。

Vector Store 提供的查詢方法有很多種玩法：
* **同步與非同步**：看你的程式架構，要跑 Sync 還是 Async 都可以。
* **查字串或查向量**：你可以直接丟一段文字（String）去查，也可以直接丟轉換好的向量（Vector）去查。
* **要不要分數**：你可以選擇回傳結果時，要不要順便附上「相似度分數」（Similarity scores，看看這結果跟你的問題有多像）。
* **相似度 vs. MMR**：最基本就是用「相似度」來查，但你也可以選擇用 **「最大邊際相關性」(Maximum Marginal Relevance, MMR)**。
    > MMR 的用途是幫你在「跟問題很像」以及「結果要多樣化」這兩者之間取得平衡，避免查出來的結果每一條都長得一模一樣，可以保留一點多樣性。

基本上，不管用哪種方法，回傳的輸出結果通常都會是一串 `Document` 物件的列表（List）。

In [10]:
from langchain_core.vectorstores import InMemoryVectorStore

vector_store = InMemoryVectorStore(embeddings)

In [11]:
ids = vector_store.add_documents(documents=all_splits)

In [12]:
results = vector_store.similarity_search(
    "How to lose weight"
)

print(results[0])

page_content='your diet enough to lose an extra 0.5 lb per week. In calories this is 3500 x 0.5 = 
1750 calories. To subtract 1750 more calories per week 1750 ​÷ ​7 = 250 calories 
per day need to be dropped. 
 
4. Continue to assess weight changes across weeks and adjust when your desired pace of 
loss or gain slows on average across two or more weeks.  
 
 
Tips for Adjusting Calories via Macronutrients  
 
Calorie changes are created by increasing or decreasing the amount(s) of one or more 
macronutrients. Merely adjusting calories up or down without regard for macronutrient amounts 
would still result in weight changes, but body composition would not necessarily change as 
desired. For example, if calories were cut evenly from all macros to create a caloric deficit, the 
deficit would certainly result in tissue loss, but might cut protein below minimum levels resulting 
in muscle loss. Considering which macros to adjust when making adjustments to your diet is' metadata={'producer':

In [13]:
# Note that providers implement different scores; the score here
# is a distance metric that varies inversely with similarity.

results = vector_store.similarity_search_with_score("How to gain weight")
doc, score = results[0]
print(f"Score: {score}\n")
print(doc)

Score: 0.5509548549722539

page_content='2. Follow adjustment steps from the muscle gain diet section. 
 
Weight gain and loss is a lot like launching a guided rocket to a target. The initial launch points 
the rocket in the right general direction, and then small adjustments keep the rocket on track 
from there. Nutrition is a bit like rocket science after all!  
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
145' metadata={'producer': 'iOS Version 16.3 (Build 20D47) Quartz PDFContext', 'creator': 'PyPDF', 'creationdate': "D:20230222113409Z00'00'", 'moddate': "D:20230222113409Z00'00'", 'source': 'example_data/diet.pdf', 'total_pages': 272, 'page': 151, 'page_label': '152', 'start_index': 743}


In [14]:
results

[(Document(id='23c9e091-f658-45d0-9a16-6527d19ec2eb', metadata={'producer': 'iOS Version 16.3 (Build 20D47) Quartz PDFContext', 'creator': 'PyPDF', 'creationdate': "D:20230222113409Z00'00'", 'moddate': "D:20230222113409Z00'00'", 'source': 'example_data/diet.pdf', 'total_pages': 272, 'page': 151, 'page_label': '152', 'start_index': 743}, page_content='2. Follow adjustment steps from the muscle gain diet section. \n \nWeight gain and loss is a lot like launching a guided rocket to a target. The initial launch points \nthe rocket in the right general direction, and then small adjustments keep the rocket on track \nfrom there. Nutrition is a bit like rocket science after all!  \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n145'),
  0.5509548549722539),
 (Document(id='ff962bb1-e705-4e9e-9b9d-3b96414b6613', metadata={'producer': 'iOS Version 16.3 (Build 20D47) Quartz PDFContext', 'creator': 'PyPDF', 'creationdate': "D:20230222113409Z00'00'", 'moddate': "D:20230222113409Z

#### 4. Retrievers (檢索器)

這邊要講一個稍微技術一點，但是很重要的觀念：**VectorStore 跟 Retriever 的差別**。

首先，剛剛講的 `VectorStore` 物件，它本身其實**不是**一個 `Runnable`（可執行物件）。

但 LangChain 的 **`Retrievers` (檢索器)** 卻是 `Runnable`。這有什麼好處？
因為它是 `Runnable`，所以它就具備了一套標準的「SOP 方法」，像是同步與非同步的 `invoke`（呼叫）、`batch`（批次處理）等等。這讓它很容易跟 LangChain 的其他元件串接在一起。

雖然我們通常是把 `VectorStore` 包裝一下變成 `Retriever` 來用，但 `Retriever` 的能力其實更廣：
* 它不一定只能查向量資料庫。
* 它也可以去接其他的資料來源，例如**外部 API**（像是去查 Google 搜尋結果、維基百科等等），只要能「撈回文件」的都可以叫 Retriever。

如果我們不想搞那麼複雜去繼承整個 `Retriever` 類別，其實也可以自己手刻一個簡易版。
只要決定好你想用哪一種方法來撈資料（例如最簡單的 `similarity_search`），我們就能很輕鬆地把它變成一個 Runnable 來用。

下面示範怎麼圍繞著 `similarity_search` 方法來打造一個簡單的檢索器：

In [15]:
from typing import List

from langchain_core.documents import Document
from langchain_core.runnables import chain


@chain
def retriever(query: str) -> List[Document]:
    return vector_store.similarity_search(query, k=1)


retriever.batch(
    [
        "How many foods are healthy?",
        "When to eat?",
    ],
)

[[Document(id='2baaaf08-2bab-40ec-9245-4c3852197039', metadata={'producer': 'iOS Version 16.3 (Build 20D47) Quartz PDFContext', 'creator': 'PyPDF', 'creationdate': "D:20230222113409Z00'00'", 'moddate': "D:20230222113409Z00'00'", 'source': 'example_data/diet.pdf', 'total_pages': 272, 'page': 222, 'page_label': '223', 'start_index': 0}, page_content='Superfoods \n \nSuperfoods are supposed to have so many nutrients in them that they can wildly affect your \nhealth by themselves. It is true that some foods really are packed with more nutrients, than \nothers, on average. Brightly colored fruits and deeply colored vegetables likely top the list, and \neven whole eggs could be classified as nutrient-dense. Nutrient density is not always desirable. \nConsider fat-soluble vitamins, for example. These can be stored in body fat and if chronically \nover-eaten can cause health problems. While it is unlikely that you will overeat “superfoods” to \nthe point of danger, their benefits are also grea

其實 `VectorStore` 本身就有內建一招 **`as_retriever`** 方法，可以直接把當前的 VectorStore 轉換成一個 `Retriever` 物件（精確一點說，是產生一個 `VectorStoreRetriever`）。

這個轉換出來的 Retriever，裡面有兩個屬性超重要，專門用來控制查詢行為：

* **`search_type`**：
    決定要用哪一種演算法去查。是要用基本的「相似度搜尋」，還是要用剛剛提過的 MMR？
* **`search_kwargs`**：
    這是用來**塞參數**的字典（dict）。就像是在設定：「我要查前幾名？（`k`值）」或是「過濾條件是什麼？」。

簡單來說，這兩個參數就是在指揮底層的 Vector Store：**「欸，等一下用什麼招式查？參數要帶多少？」**。

舉個例子，我們可以用下面這行程式碼，直接重現剛剛上面講的功能（而且更簡單）：

In [16]:
retriever = vector_store.as_retriever(
    search_type="similarity",
    search_kwargs={"k": 1},
)

retriever.batch(
    [
        "How to be healthy",
        "How to become a millionaire",
    ],
)

[[Document(id='4a9fad47-553b-4552-ad00-cc59859bce6c', metadata={'producer': 'iOS Version 16.3 (Build 20D47) Quartz PDFContext', 'creator': 'PyPDF', 'creationdate': "D:20230222113409Z00'00'", 'moddate': "D:20230222113409Z00'00'", 'source': 'example_data/diet.pdf', 'total_pages': 272, 'page': 207, 'page_label': '208', 'start_index': 3131}, page_content='protocol. In order to restore your health to the condition you had before a stint of poor eating \npractices, you have to eat well for \u200bat least as long\u200b as you ate badly. In some cases eating badly \ncan do irreversible damage to your health, so buying into quick fixes can be even more \n201')],
 [Document(id='10b7757f-ecee-4956-aa08-d9438c0b8f4f', metadata={'producer': 'iOS Version 16.3 (Build 20D47) Quartz PDFContext', 'creator': 'PyPDF', 'creationdate': "D:20230222113409Z00'00'", 'moddate': "D:20230222113409Z00'00'", 'source': 'example_data/diet.pdf', 'total_pages': 272, 'page': 234, 'page_label': '235', 'start_index': 1625}

#### 5. RAG agents (RAG 代理人)

最後我們來聊聊 **RAG Agent**。

其實 RAG 應用程式有一種很常見的玩法，就是把它設計成一個簡單的 **Agent (代理人)**，然後給這個代理人配備一個「能撈資料的工具」。

我們可以組裝一個最陽春、最小型的 RAG Agent，做法超簡單：
就是寫一個 Tool（工具），把我們剛剛做好的 **Vector Store 給「包起來」**。

這樣一來，這個 Agent 就具備了查詢能力，當它覺得需要查資料來回答問題時，就會自己去呼叫這個工具來用。

In [17]:
from langchain.tools import tool

@tool(response_format="content_and_artifact")
def retrieve_context(query: str):
    """Retrieve information to help answer a query."""
    retrieved_docs = vector_store.similarity_search(query, k=2)
    serialized = "\n\n".join(
        (f"Source: {doc.metadata}\nContent: {doc.page_content}")
        for doc in retrieved_docs
    )
    return serialized, retrieved_docs

In [ ]:
from langchain.agents import create_agent


tools = [retrieve_context]
# If desired, specify custom instructions
prompt = (
    "You have access to a tool that retrieves context from a blog post. "
    "Use the tool to help answer user queries."
)
## TODO: agent = 

In [19]:
query = (
    "有什麼方法可以增肌？"
    "等你拿到答案之後，再去查查看這個方法還有哪些常見的延伸用法，請用繁體中文回答。"
)

for event in agent.stream(
    {"messages": [{"role": "user", "content": query}]},
    stream_mode="values",
):
    event["messages"][-1].pretty_print()

================================ Human Message =================================

有什麼方法可以增肌？等你拿到答案之後，再去查查看這個方法還有哪些常見的延伸用法，請用繁體中文回答。
================================== Ai Message ==================================
Tool Calls:
  retrieve_context (call_VhkDVkRalvLNDyofT7pof64h)
 Call ID: call_VhkDVkRalvLNDyofT7pof64h
  Args:
    query: 增肌的方法
================================= Tool Message =================================
Name: retrieve_context

Source: {'producer': 'iOS Version 16.3 (Build 20D47) Quartz PDFContext', 'creator': 'PyPDF', 'creationdate': "D:20230222113409Z00'00'", 'moddate': "D:20230222113409Z00'00'", 'source': 'example_data/diet.pdf', 'total_pages': 272, 'page': 183, 'page_label': '184', 'start_index': 756}
Content: appear as large and defined as possible, with visible vascularization under the skin. When you 
manipulate glycogen stores by increasing carbohydrate intake, you will fill your stores making 
your muscles look full, but also risk water retention under the skin w